In [ ]:
'''This code cell is for preprocessing image data from google drive zip folder'''
!gdown https://drive.google.com/uc?id=1Lbk_pwATorDcUyk9VcGYzomLR4Rhj-lv #Downloading zip file from google drive
zip_path = '/content/Cat_GAN_1.zip' #Getting the path 
!unzip -q Cat_GAN_1.zip #Unzipping the folder
!rm Cat_GAN_1.zip #Removing the zip folder
import os #Imported required libraries for this step
import pdb
import cv2
import operator
from PIL import Image
os.mkdir("Training") #making the new Training and testing folders
dirs = ["Domino", "Stormy"] #Making the names for the subdirec
pardirs = ["/content/Training"]
for pardir in pardirs:
  for dir in dirs: 
    path = os.path.join(pardir, dir)
    os.mkdir(path)
datadir1 = "/content/Cat_Classifier_Images_70-30 /Train/Domino"
datadir2 = "/content/Cat_Classifier_Images_70-30 /Train/Stormy"
datadir3 = "/content/Cat_Classifier_Images_70-30 /Test/Domino"
datadir4 = "/content/Cat_Classifier_Images_70-30 /Test/Stormy"
filelist1 = sorted(os.listdir(datadir1), key = lambda fname: int(fname.split("_")[0][-4:]))
filelist2 = sorted(os.listdir(datadir2), key = lambda fname: int(fname.split("_")[0][-4:]))
filelist3 = sorted(os.listdir(datadir3), key = lambda fname: int(fname.split("_")[0][-4:]))
filelist4 = sorted(os.listdir(datadir4), key = lambda fname: int(fname.split("_")[0][-4:]))
datadirs = [filelist1, filelist2, filelist3, filelist4]
inc = 0
idom = 0
istorm = 0
for filelist in datadirs:
  for fil in filelist:
    if inc == 0:
      path = "/content/Cat_Classifier_Images_70-30 /Train/Domino/" + fil
      idom += 1
      img = cv2.imread(path)
      imgResized = cv2.resize(img, (48, 64))
      cv2.imwrite('/content/Training/Domino/DominoTR%03i.jpg' %idom, imgResized)
    elif inc == 1:
      path = "/content/Cat_Classifier_Images_70-30 /Train/Stormy/" + fil
      istorm += 1
      img = cv2.imread(path)
      imgResized = cv2.resize(img, (48, 64))
      cv2.imwrite('/content/Training/Stormy/StormyTR%03i.jpg' %istorm, imgResized)
    elif inc == 2:
      path = "/content/Cat_Classifier_Images_70-30 /Test/Domino/" + fil
      img = cv2.imread(path)
      idom += 1
      imgResized = cv2.resize(img, (48, 64))
      cv2.imwrite('/content/Training/Domino/DominoTR%03i.jpg' %idom, imgResized)
    else:
      path = "/content/Cat_Classifier_Images_70-30 /Test/Stormy/" + fil
      img = cv2.imread(path)
      istorm += 1
      imgResized = cv2.resize(img, (48, 64))
      cv2.imwrite('/content/Training/Stormy/StormyTR%03i.jpg' %istorm, imgResized)
  inc += 1

In [ ]:
import pdb
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import os
datadirDom = '/content/Training/Domino'
datadirStorm = '/content/Training/Stormy'
datadirDom = os.listdir(datadirDom)
datadirStorm = os.listdir(datadirStorm)
labels = []
cat_images = []
for fil in datadirDom:
  path = '/content/Training/Domino/' + fil
  image = Image.open(path)
  horz_image = np.array(image.transpose(method = Image.FLIP_LEFT_RIGHT))
  vert_image = np.array(image.transpose(method = Image.FLIP_TOP_BOTTOM))
  rot_image = np.array(image.rotate(180))
  image = np.array(image)
  cat_images.append(image)
  plt.imshow(image)
  cat_images.append(horz_image)
  cat_images.append(vert_image)
  cat_images.append(rot_image)
for fil in datadirStorm:
  path = '/content/Training/Stormy/' + fil
  image = Image.open(path)
  horz_image = np.array(image.transpose(method = Image.FLIP_LEFT_RIGHT))
  vert_image = np.array(image.transpose(method = Image.FLIP_TOP_BOTTOM))
  rot_image = np.array(image.rotate(180))
  image = np.array(image)
  cat_images.append(image)
  cat_images.append(horz_image)
  cat_images.append(vert_image)
  cat_images.append(rot_image)
cat_images = np.array(cat_images)
print (cat_images.shape)

In [ ]:
from tensorflow.keras.layers import Activation, Dense, Input
from tensorflow.keras.layers import Conv2D, Flatten
from tensorflow.keras.layers import Reshape, Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import concatenate
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
import numpy as np
import math
import matplotlib.pyplot as plt
import os
import argparse
import pdb
def build_generator(inputs, image_size):
    image_resize = (image_size[0] // 4, image_size[1] // 4)
    kernel_size = 5
    layer_filters = [128, 64, 32, 3]
    x = inputs
    x = Dense(image_resize[0] * image_resize[1] * 3 * layer_filters[0])(x)
    x = Reshape((image_resize[0], image_resize[1], 3 * layer_filters[0]))(x)
    for filters in layer_filters:
        if filters > layer_filters[-2]:
            strides = 2
        else:
            strides = 1
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Conv2DTranspose(filters=filters, kernel_size=kernel_size, strides=strides, padding='same')(x)
    x = Reshape((64, 48, 3, 1))(x)
    x = Activation('sigmoid')(x)
    generator = Model(inputs, x, name='generator')
    return generator
def build_discriminator(inputs, image_size):
    kernel_size = 5
    layer_filters = [32, 64, 128, 256]
    x = inputs
    for filters in layer_filters:
        if filters == layer_filters[-1]:
            strides = 1
        else:
            strides = 2
        x = LeakyReLU(alpha=0.2)(x)
        x = Conv2D(filters = filters, kernel_size = kernel_size, strides = strides, padding = 'same')(x)
    x = Flatten()(x)
    x = Dense(1)(x)
    x = Activation('sigmoid')(x)
    discriminator = Model(inputs, x, name='discriminator')
    return discriminator
def train(models, data, params):
    losss = []
    accc = []
    generator, discriminator, adversarial = models
    x_train = data
    batch_size, latent_size, train_steps, model_name = params
    save_interval = 500
    noise_input = np.random.uniform(-1.0, 1.0, size=[16, latent_size])
    train_size = x_train.shape[0]
    accavg = 0
    accavg1 = 0
    epochs = [i for i in range(train_steps)]
    for i in range(train_steps):
        rand_indexes = np.random.randint(0, train_size, size=batch_size)
        real_images = x_train[rand_indexes]
        noise = np.random.uniform(-1.0, 1.0, size=[batch_size, latent_size])
        fake_images = generator.predict(noise)
        x = np.concatenate((real_images, fake_images))
        y = np.ones([2 * batch_size, 1])
        y[batch_size:, :] = 0.0
        loss, acc = discriminator.train_on_batch(x, y)
        losss.append(loss)
        accc.append(acc)
        accavg += acc
        log = "%d: [discriminator loss: %f, acc: %f]" % (i, loss, acc)
        noise = np.random.uniform(-1.0, 1.0, size=[batch_size, latent_size])
        y = np.ones([batch_size, 1])
        loss, acc = adversarial.train_on_batch(noise, y)
        accavg1 += acc
        log = "%s [adversarial loss: %f, acc: %f]" % (log, loss, acc)
#        print(log)
        if (i + 1) % save_interval == 0:
           accavg = accavg / save_interval
           print ("Average discriminator accuracy: " + str(accavg))
           accavg1 = accavg1 / save_interval
           print ("Average adversarial accuracy: " + str(accavg1))
           accavg = 0
           accavg1 = 0
           plot_images(generator, noise_input, show = False, step = i + 1)
           images = generator.predict(noise_input)
    plt.plot(epochs, losss)
    plt.title("Discriminator loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.show()
    plt.savefig('DCGAN_discriminator_loss.jpg')
    plt.plot(epochs, accc)
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.title("Discriminator Accuracy")
    plt.show()
    plt.savefig('DCGAN_discriminator?accuracy.jpg')
    generator.save(model_name + ".h5")
def plot_images(generator, noise_input, show=False, step=0, model_name="gan"):
    os.makedirs(model_name, exist_ok=True)
    filename = os.path.join(model_name, "%05d.png" % step)
    images = generator.predict(noise_input)
    plt.figure(figsize=(11.1, 11.1))
    num_images = images.shape[0]
    image_size = images.shape[1]
    rows = int(math.sqrt(noise_input.shape[0]))
    for i in range(num_images):
        plt.subplot(rows, rows, i + 1)
        plt.imshow(np.array(images[i - 1]).reshape((64, 48, 3)))
        plt.axis('off')
    plt.savefig(filename)
    if show:
        plt.show()
    else:
        plt.close('all')
def build_and_train_models():
    x_train = cat_images
    x_train = np.reshape(x_train, [x_train.shape[0], x_train.shape[1], x_train.shape[2], x_train.shape[3], 1])
    x_train = x_train.astype('float32') / 255
    model_name = "cgan_cat"
    latent_size = 2
    batch_size = 64
    train_steps = 40000
    lr = 2e-4
    decay = 6e-8
    input_shape = (64, 48, 3, 1)
    label_shape = (2,)
    image_size = (64, 48, 3)
    inputs = Input(shape=input_shape, name='discriminator_input')
    discriminator = build_discriminator(inputs, image_size)
    optimizer = RMSprop(lr=lr, decay=decay)
    discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    discriminator.summary()
    input_shape = (latent_size, )
    inputs = Input(shape=input_shape, name='z_input')
    generator = build_generator(inputs, image_size)
    generator.summary()
    optimizer = RMSprop(lr=lr*0.5, decay=decay*0.5)
    discriminator.trainable = False
    outputs = discriminator(generator(inputs))
    adversarial = Model(inputs, outputs, name=model_name)
    adversarial.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    adversarial.summary()
    models = (generator, discriminator, adversarial)
    data = x_train
    params = (batch_size, latent_size, train_steps, model_name)
    train(models, data, params)
build_and_train_models()